In [72]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import warnings
warnings.filterwarnings("ignore")

In [73]:
from torchvision import datasets, transforms

In [74]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [75]:
import os

from pathlib import Path
fileDir = os.path.dirname(os.path.realpath('__file__'))
train_path= fileDir+'/datasets/data/Train_DS'
test_path= fileDir+'/datasets/data/Test_DS'

train_path = Path(train_path)
test_path = Path(test_path)
print(train_path)


C:\Users\ravik\Ravi\Projects\Gold_ImageSimilarity\datasets\data\Train_DS


In [76]:
train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

In [77]:
class_types = train_dataset.classes
print(class_types)

['Cat', 'Dog']


In [78]:
train_dataset.samples
new_class=[]

for index,animal_type in enumerate(class_types):
    animal_path = os.path.join(train_path, animal_type)
    #print(animal_path)
    for breed in os.listdir(animal_path):
        # Normalize breed name
        breed_normalized = breed.lower().strip()
        new_class.append(animal_type+"/"+breed_normalized)
        
        #print(class_types)

print(new_class)

#train_dataset.classes=new_class
#test_dataset.classes=new_class

['Cat/british', 'Cat/persian', 'Dog/affenpinscher', 'Dog/afghanhound']


In [79]:
# values = [i for i,c in enumerate(train_dataset.classes)]
# res = dict(zip(values, train_dataset.classes))
 
# train_dataset.class_to_idx=res
# test_dataset.class_to_idx=res

In [80]:
train_dataset.class_to_idx



{'Cat': 0, 'Dog': 1}

In [81]:
class_types = train_dataset.classes
print(class_types)

['Cat', 'Dog']


In [82]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [83]:
train_dataset.classes

['Cat', 'Dog']

In [84]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224", num_labels=len(train_dataset.classes), ignore_mismatched_sizes=True)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-5)

In [86]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(5):  # Adjust the number of epochs as needed
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.3917563259601593
Epoch 2, Loss: 0.21703286468982697
Epoch 3, Loss: 0.10387345403432846
Epoch 4, Loss: 0.050606485456228256
Epoch 5, Loss: 0.039616189897060394


In [87]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Accuracy: 100.00%


In [88]:
train_dataset.classes[1].split('/')[0]

'Dog'

In [89]:
train_dataset.classes

['Cat', 'Dog']

In [95]:
def predict(image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB format
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        outputs = model(image).logits
        print(outputs)
        predicted_class_idx = outputs.argmax(-1).item()

    print(predicted_class_idx)
    # Get the predicted breed from the dataset classes
    breed = train_dataset.classes[predicted_class_idx]  # This should return "Animal/Breed"
    print(breed)
    # Safely split the breed label
    split_breed = breed.split('/')

    print(split_breed)
    #print(breed_name)

    if len(split_breed) == 2:  # Expecting "Animal/Breed"
        animal_type = split_breed[0]
        breed_name = split_breed[1]
        return f"Animal is {animal_type} and it belongs to breed '{breed_name}'"
    else:
        return "Predicted class format is unexpected."

In [98]:
result = predict(r'test_P3_cat.jpg')

print(result)

tensor([[ 0.2448, -0.8232]])
0
Cat
['Cat']
Predicted class format is unexpected.


---